In [3]:
import torch
import pandas as pd
from kobert_transformers import get_kobert_model, get_tokenizer
import torch.nn as nn
import re

# ===== 1️⃣ 모델 불러오기 =====
MODEL_PATH = "model_test/kobert_emotion_model/model.pt"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_labels = 3
max_len = 64
dropout_rate = 0.3

class KoBERTEmotionClassifier(nn.Module):
    def __init__(self, bert, num_labels, dropout=0.3):
        super().__init__()
        self.bert = bert
        self.drop = nn.Dropout(dropout)
        self.fc = nn.Linear(768, num_labels)
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.pooler_output if hasattr(outputs, "pooler_output") else outputs[1]
        logits = self.fc(self.drop(pooled))
        return logits

tokenizer = get_tokenizer()
bert = get_kobert_model()
model = KoBERTEmotionClassifier(bert, num_labels=num_labels, dropout=dropout_rate).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

# ===== 2️⃣ 예시 자기소개서 =====
text = """SpringBoot를 활용해 공부를 하려했지만 한계에 부딪혔습니다.그래서 매일 밤을 새며 공부를 하며 극복했습니다."""

# ===== 3️⃣ 문장 단위로 분리 =====
def split_sentences_korean(text):
    # 마침표/물음표/느낌표 뒤에서 공백 있든 없든 분리
    sents = re.split(r'(?<=[.!?])(?:\s+)?', text.strip())
    return [s.strip() for s in sents if len(s.strip()) > 3]


sentences = split_sentences_korean(text)
print(f"총 {len(sentences)}문장 분석\n")
print("✅ GPU 사용 여부:", torch.cuda.is_available())
print("✅ 현재 디바이스:", torch.cuda.get_device_name(0))
print("✅ 현재 GPU 메모리 사용량:", torch.cuda.memory_allocated(0)/1024**2, "MB")

# ===== 4️⃣ 문장별 감정 추론 =====
results = []
label_map = {0: "부정", 1: "중립", 2: "긍정"}

for sent in sentences:
    enc = tokenizer(
        sent,
        truncation=True,
        padding="max_length",
        max_length=max_len,
        return_tensors="pt"
    )
    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device)

    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]
        pred_label = int(probs.argmax())

    results.append({
        "문장": sent,
        "예측라벨": label_map[pred_label],
        "부정확률": round(probs[0], 4),
        "중립확률": round(probs[1], 4),
        "긍정확률": round(probs[2], 4)
    })

df = pd.DataFrame(results)
print(df)


총 2문장 분석

✅ GPU 사용 여부: True
✅ 현재 디바이스: Tesla V100-PCIE-32GB
✅ 현재 GPU 메모리 사용량: 352.73876953125 MB
                                      문장 예측라벨    부정확률    중립확률    긍정확률
0  SpringBoot를 활용해 공부를 하려했지만 한계에 부딪혔습니다.   부정  0.9959  0.0006  0.0035
1            그래서 매일 밤을 새며 공부를 하며 극복했습니다.   긍정  0.0012  0.0007  0.9981


In [1]:
!ls

AI  CONVENTION.md  model_test  requirements.txt  Untitled.ipynb


In [2]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!source aihubshell

aihubshell version 25.09.19 v0.6
Invalid mode. Please use 'pd','pl' for datapckage list,'d', 'l' for dataset list, 'l [datasetkey]' for file tree
